In [1]:
import numpy as np
import pyopencl as cl
import numpy.linalg as la

In [2]:
vector_dimension = 100

In [3]:
vector_a = np.random.randint(vector_dimension,size=vector_dimension) 
vector_b = np.random.randint(vector_dimension,size=vector_dimension)

In [4]:
platform = cl.get_platforms()[0]
device = platform.get_devices()[0]
context = cl.Context([device])
queue = cl.CommandQueue(context)

In [5]:
mf = cl.mem_flags 
a_g = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=vector_a) 
b_g = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=vector_b) 

In [6]:
program = cl.Program(context, """ 
__kernel void vectorSum(__global const int *a_g, __global const int *b_g, __global int *res_g) { 
  int gid = get_global_id(0); 
  res_g[gid] = a_g[gid] + b_g[gid]; 
} 
""").build()

In [7]:
res_g = cl.Buffer(context, mf.WRITE_ONLY, vector_a.nbytes)

In [8]:
program.vectorSum(queue, vector_a.shape, None, a_g, b_g, res_g)

In [9]:
res_np = np.empty_like(vector_a)

In [10]:
cl.enqueue_copy(queue, res_np, res_g)

In [11]:
print ("PyOPENCL SUM OF TWO VECTORS")
print ("Platform Selected = %s" %platform.name )
print ("Device Selected = %s" %device.name)
print ("VECTOR LENGTH = %s" %vector_dimension)
print ("INPUT VECTOR A")
print (vector_a)
print ("INPUT VECTOR B")
print (vector_b)
print ("OUTPUT VECTOR RESULT A + B ")
print (res_np)

PyOPENCL SUM OF TWO VECTORS
Platform Selected = NVIDIA CUDA
Device Selected = NVIDIA GeForce GTX 1650
VECTOR LENGTH = 100
INPUT VECTOR A
[60 58 29 73 93 97 25 93 91 14 40 89  1  3 17 76  3 58 65 65 99 13 39 87
 70 56 91 46  5 65 13 54 91  0 45 64 59 74 99 65 30 49 13 13 96 77 93 15
  5 56 79 93 95 70 99 12 92 17 63 55 47 52 23 71 71 33 57 26 20 74 31 78
  2  3 75 58 98 33 87 82 20 36 24 20 25 24  6  5 44 28 48 18 53  4 18 21
 80  9  3  7]
INPUT VECTOR B
[86 24 95 17  6 93  6 83 60 97 40 85 74 40 88  1 17 18 76 98 80  2 97 32
 79 25 49 21 17 19 49 18  9 78 67  4 26 47 74 48 99 93 13 80 99 68 67 65
  6 81 67  0 44 25 66 37 37  8 76 39 15 27 66 23 92 52 98 54 37 15 10 48
 20 79 87 85 89 54 65 80 83 46 96 14 10 37 69 11 34 74 79 92 15 57  9 70
 67  9 58  0]
OUTPUT VECTOR RESULT A + B 
[146  82 124  90  99 190  31 176 151 111  80 174  75  43 105  77  20  76
 141 163 179  15 136 119 149  81 140  67  22  84  62  72 100  78 112  68
  85 121 173 113 129 142  26  93 195 145 160  80  11 137 146  

In [12]:
assert(la.norm(res_np - (vector_a + vector_b))) < 1e-5